In [ ]:
path_for_joined_geotiff_df = '' 
path_for_folder_to_save_models = ''
path_for_tf_dataset = ''

#Number of models to be trained
search_iterations = 100
#Max Number of epochs to train each model. Most models will probably fail before reaching max number of epochs.
epochs = 100

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np
#from google.colab import drive
import pandas as pd
from osgeo import gdal
from skimage.transform import resize
#drive.mount('/content/drive')
import random

Mounted at /content/drive


In [ ]:
df = pd.read_csv(path_for_joined_geotiff_df)

In [ ]:
#These Rows (from yoesmetie) had random points with elevation in the billions, so I just exluded them
outliers = [964, 970, 975, 976, 977, 982, 983, 984, 985, 992, 993, 994, 995]
df = df[~df.index.isin(outliers)]

In [ ]:
image_size = 128
batch_size = 32

In [ ]:
#Loads Tensorflow dataset tf_dataset_128x128
dataset = tf.data.experimental.load(path_for_tf_dataset)

In [ ]:
steps_per_epoch = int(np.ceil(len(dataset)/batch_size))
num_classes = len(np.unique(df.BIOME_NAME))
latent_dim = ((16*16)*num_classes)-num_classes
generator_input = latent_dim + num_classes
discriminator_in_channels = num_channels + num_classes

In [ ]:
def build_discriminator():
    discriminator = keras.Sequential(
      [
          keras.layers.InputLayer((image_size, image_size, discriminator_in_channels)),
          layers.BatchNormalization(momentum=0.5),
          layers.Conv2D(64, (4, 4), strides=(2, 2), padding="same" 
                        , activity_regularizer=keras.regularizers.L2(.00001)),
          layers.BatchNormalization(momentum=0.5),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2D(128, (4, 4), strides=(2, 2), padding="same" 
                        , activity_regularizer=keras.regularizers.L2(.00001)),
          layers.BatchNormalization(momentum=0.5),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2D(256, (4, 4), strides=(2, 2), padding="same" 
                        , activity_regularizer=keras.regularizers.L2(.00001)),
          layers.BatchNormalization(momentum=0.5),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2D(512, (4, 4), strides=(2, 2), padding="same" ,
                        activity_regularizer=keras.regularizers.L2(.00001)),
          layers.BatchNormalization(momentum=0.5),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2D(1024, (3, 3), strides=(2, 2), padding="same", 
                         activity_regularizer=keras.regularizers.L2(.00001)),
          layers.LeakyReLU(alpha=0.2),
          layers.GlobalMaxPooling2D(),
          layers.Dense(1, activation='sigmoid')
      ],
      name="discriminator",
  )
  return discriminator

In [ ]:
class ConditionalGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(ConditionalGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")

    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker]

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(ConditionalGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, data):
        #load data
        real_images, one_hot_labels = data

        # Add dimension for the labels
        image_one_hot_labels = one_hot_labels[:, :, None, None]
        image_one_hot_labels = tf.repeat(
            image_one_hot_labels, repeats=[image_size * image_size]
        )
        image_one_hot_labels = tf.reshape(
            image_one_hot_labels, (-1, image_size, image_size, num_classes)
        )

        # Generate random points in the latent space.
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        #Add the labels
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )
        #Generate fake terrain with generator
        generated_images = self.generator(random_vector_labels)

        #Combine terrain with labels
        fake_image_and_labels = tf.concat([generated_images, image_one_hot_labels], -1)
        real_image_and_labels = tf.concat([real_images, image_one_hot_labels], -1)
        #Real and Fake Terrain
        combined_images = tf.concat(
            [fake_image_and_labels, real_image_and_labels], axis=0
        )

        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )

        # Train the discriminator.
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )
        
        #Sample random points in the latent space.
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Assemble labels that say "all real images".
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_vector_labels)
            fake_image_and_labels = tf.concat([fake_images, image_one_hot_labels], -1)
            predictions = self.discriminator(fake_image_and_labels)
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        #Track loss
        self.gen_loss_tracker.update_state(g_loss)
        self.disc_loss_tracker.update_state(d_loss)

        return{
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
        }

In [ ]:
#If true then generator's first layer is a dense layer
dense = [True, False]
#Number of conv2d layers in generator
conv2d_layers = [i for i in range(3,8)]
#Alpha used in generator
alpha = [.01, .1, .2, .4, .8]
#l2 used in generator
l2 = [1*10**-i for i in range(1,7)]
#first filter size
filter_size = [2**i for i in range(11,1,-1)]
#Determines how filter size will change from the first filter size
filter_growth = ['decreasing', 'constant', 'increasing']
#first kernel size
kernel_size = [2**i for i in range(2,7)]
##Determines how kernel size will change from the first kernel size
kernel_growth = ['decreasing', 'constant', 'increasing']
#possible momemtum in generator
momentum = [.8, .5, .3, .1, .05]
#generator's possible learning ratae
learning_rate = [.1, .05, .01, .005, .001, .0005, .0001, .00005, .00001]
#possible steps per learning rate decay
steps_per_lr_decay = [1, steps_per_epoch//5, steps_per_epoch//2, steps_per_epoch, steps_per_epoch*2, steps_per_epoch*5]
#Dictionary of the all the lists of possible params
possible_params = {'dense': dense, 'conv2d_layers': conv2d_layers, 'alpha': alpha, 
                     'l2':l2, 'filter_size': filter_size, 'filter_growth': filter_growth,'kernel_size':kernel_size, 'kernel_growth':kernel_growth, 
                     'momentum':momentum, 'learning_rate': learning_rate, 'steps_per_lr_decay':steps_per_lr_decay}

In [ ]:
#Randomly chooses params from param space adn returns dictionary of choosen params
def get_params(param_space):
    choosen_params = {}
    #loops through each param
    for param_name, param in param_space.items():
        #chooses random value from list of possible value for each parameter
        choosen_params[param_name] = random.choice(param)
    return choosen_params

In [1]:
#Returns list of kernel and filters sizes for each conv2d_layer
def get_kernels_and_filters(first_filter_size, filter_growth, first_kernel_size, kernel_growth, conv2d_layers):
  #Used to constrain the size of the model when increasing and decreasing growth for kernel and filter size
  max_filter = 2048
  min_filter = 4
  max_kernel = 32
  min_kernel = 4
  if filter_growth == 'constant':
    #All filter sizes will be the same size as the first filter
    filters = [first_filter_size for i in range(conv2d_layers)]
  elif filter_growth == 'increasing':
    #filter sizes will be first_filter_size*(2^i) for i=0 to i = conv2d_layers
    filters = [first_filter_size*2**i for i in range(conv2d_layers)]
  else:
    ##filter sizes will be first_filter_size*(2^-i) for i=0 to i = conv2d_layers
    filters = [first_filter_size*2**-i for i in range(conv2d_layers)]  
  if kernel_growth == 'constant':
    #All kernel sizes will be the same size as the first kernel
    kernels = [first_kernel_size for i in range(conv2d_layers)]
  elif kernel_growth == 'increasing':
    #kernel sizes will be first_kernel_size*(2^i) for i=0 to i = conv2d_layers
    kernels = [first_kernel_size*2**i for i in range(conv2d_layers)]
  else:
    #kernel sizes will be first_kernel_size*(2^-i) for i=0 to i = conv2d_layers
    kernels = [first_kernel_size*2**-i for i in range(conv2d_layers)]
  kernels =  np.asarray(kernels)
  filters = np.asarray(filters)
  #if a filter/kernel size is greater/less than max/min than it is set to the max/min
  kernels[kernels < min_kernel] = min_kernel
  kernels[kernels > max_kernel] = max_kernel
  filters[filters > max_filter] = max_filter
  filters[filters < min_filter] = min_filter
  return kernels, filters

In [ ]:
#returns list of strides for each conv2d layer
def get_strides(conv2d_layers):
  #Every models last two conv2d layers will have (2,2) strides
  strides = [(2,2), (2,2)]
  for i in range(conv2d_layers-3):
    strides.insert(0,(1,1))
  return strides

In [ ]:
#Stops training if discriminator loss or generator loss gets close to zero
class Custom_Callback(keras.callbacks.Callback):
   def on_epoch_end(self, epoch, logs=None): 
         if (logs.get('d_loss') < .0001) or (logs.get('g_loss') < .0001):
           self.model.stop_training = True

In [ ]:
#Builds generator with the random params
def build_generator(iter, choosen_params):
  strides = get_strides(choosen_params['conv2d_layers'])
  kernels, filters = get_kernels_and_filters(first_filter_size=choosen_params['filter_size'], filter_growth=choosen_params['filter_growth'], 
                          first_kernel_size=choosen_params['kernel_size'], kernel_growth=choosen_params['kernel_growth'], conv2d_layers=choosen_params['conv2d_layers'])
  generator = keras.Sequential(name="generator" + str(iter))
  generator.add(keras.layers.InputLayer(generator_input,))
  #Maps latent space to 2d space using dense layers
  if choosen_params['dense']:
    generator.add(layers.Dense(16*16*num_classes, activity_regularizer=keras.regularizers.L2(choosen_params['l2'])))
    generator.add(layers.LeakyReLU(alpha=choosen_params['alpha']))
    generator.add(layers.BatchNormalization(momentum=choosen_params['momentum']))
    generator.add(layers.Reshape((16, 16, num_classes)))
  else:
    #If dense layer isn't used then latent space is reshaped to 2d
    generator.add(layers.Reshape((16, 16, num_classes)))
  for i in range(choosen_params['conv2d_layers']-1):
    generator.add(layers.Conv2DTranspose(filters[i], (int(kernels[i]), int(kernels[i])),
                                           padding='same', activity_regularizer=keras.regularizers.L2(choosen_params['l2']), strides=strides[i]))
    generator.add(layers.LeakyReLU(alpha=choosen_params['alpha']))
    if i != choosen_params['conv2d_layers'] - 1:
      generator.add(layers.BatchNormalization(momentum=choosen_params['momentum']))
  generator.add(layers.Conv2DTranspose(1, (int(kernels[-1]), int(kernels[-1])),
                                           padding='same', activity_regularizer=keras.regularizers.L2(choosen_params['l2']), strides=(2,2)))
  generator.add(layers.LeakyReLU(alpha=choosen_params['alpha']))
  return generator

In [ ]:
#Saves Models that train for all the epochs and returns dataframe of model paramters and results
def random_search(iter, epochs, param_space):
  for i in range(iter):
    params_df = pd.DataFrame(columns=['tuner_iter','g_loss', 'd_loss', 'epochs_before_failure','dense','conv2d_layers', 
                                      'alpha', 'l2', 'filter_size', 'filter_growth','kernel_size', 'kernel_growth' ,'momentum', 'learning_rate', 'steps_per_lr_decay'])
    #Gets random params from param space
    params = get_params(param_space)

    #Builds generator with random parameters
    generator = build_generator(iter, params)
    
    #builds discriminator with same params every time
    discriminator = build_discriminator()

    #generator learning rate is random, discriminator learning rate is constant
    g_initial_learning_rate = params['learning_rate']
    d_initial_learning_rate = 0.001
    #Learning rate decays at rate initial_learning_rate * .9986^(decay_steps/steps_per_epoch)
    #generator learning rate decay steps is random
    g_lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
      g_initial_learning_rate,
      decay_steps=params['steps_per_lr_decay'],
      decay_rate=.9986
      )
  
    d_lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    d_initial_learning_rate,
    decay_steps=steps_per_epoch,
    decay_rate=.9986
    )
    #creates condtional gan
    cond_gan = ConditionalGAN(
      discriminator=discriminator, generator=generator, latent_dim=latent_dim)
    
    cond_gan.compile(
      d_optimizer=keras.optimizers.Adam(learning_rate=d_lr_schedule),
      g_optimizer=keras.optimizers.Adam(learning_rate=g_lr_schedule),
      loss_fn=keras.losses.BinaryCrossentropy(),
    )

    custom_checkpoint = Custom_Callback()

    history = cond_gan.fit(dataset, epochs=epochs, callbacks=[custom_checkpoint], verbose=2)

    #adds params and training results to params_df
    params['tuner_iter'] = i
    params['g_loss'] = history.history['g_loss'][-1]
    params['d_loss'] = history.history['d_loss'][-1]
    params['epochs_before_failure'] = len(history.history['d_loss'])
    params_df = params_df.append(params, ignore_index=True)
    

    #if the model trained for the full number of epochs without failing then the model is saved
    if len(history.history['g_loss']) == epochs:
      generator.save(path_for_folder_to_save_models + '/generator_' + str(i))
    
    #models are deleted
    del generator
    del discriminator
    del cond_gan
    keras.backend.clear_session()
  return params_df

In [ ]:
params_df = random_search(search_iterations, epochs, possible_params)

ResourceExhaustedError: ignored

In [ ]:
params_df.to_csv(path_for_folder_to_save_models + '/params_df.csv')